# Importing the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import re
import string

# Inserting the dataset

In [2]:
df_data = pd.read_csv("news.csv")
df_data.head(5)

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
#dropping the columns not required
df_data = df_data.drop(['Unnamed: 0','title'],axis=1)
df_data.head(5)

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [4]:
#Finding null values
df_data.isnull().sum()

text     0
label    0
dtype: int64

# Cleaning the Data 

In [5]:
#create a function that removes special characters, urls, extra spaces and lowercase
def word_cleaner(text):
     # Remove all the special characters
    text = re.sub(r'\W', ' ',text)

    # remove all single characters
    text= re.sub(r'\s+[a-zA-Z]\s+', ' ',text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ',text) 

    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)

    # Converting to Lowercase
    text = text.lower()
    return text

In [6]:
df_data["text"] = df_data["text"].apply(word_cleaner)
df_data

,text,label
0,daniel greenfield shillman journalism fellow a...,FAKE
1,google pinterest digg linkedin reddit stumbleu...,FAKE
2,u secretary of state john kerry said monday th...,REAL
3,kaydee king kaydeeking november 9 2016 the le...,FAKE
4,it primary day in new york and front runners h...,REAL
...,...,...
6330,the state department told the republican natio...,REAL
6331,the in pbs should stand for plutocratic or pen...,FAKE
6332,anti trump protesters are tools of the oligar...,FAKE
6333,addis ababa ethiopia president obama convened ...,REAL


## Defining the target and features and feature engineering


In [7]:
X = df_data['text']
y = df_data['label']

In [8]:
#One hot encoding the categorical df
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 1, ..., 0, 1, 1])

In [9]:
#Tokenizatin and stopwords
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=2500,min_df=7,max_df=0.8, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(X).toarray()

In [10]:
#Dividing the data to train and test sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size =0.2,random_state = 0)

# Training and finding accuracy from different techniques

# 1. Logistic regression

In [11]:
from sklearn.linear_model import LogisticRegression

In [12]:
log_reg = LogisticRegression()
log_reg.fit(X_train,y_train)

LogisticRegression()

In [13]:
log_pred = log_reg.predict(X_test)

In [14]:
log_reg.score(X_test,y_test)

0.9068666140489345

In [15]:
print(classification_report(y_test,log_pred))

              precision    recall  f1-score   support

           0       0.91      0.90      0.90       615
           1       0.91      0.91      0.91       652

    accuracy                           0.91      1267
   macro avg       0.91      0.91      0.91      1267
weighted avg       0.91      0.91      0.91      1267



# 2. Random Forest Classifier

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
fr_clf = RandomForestClassifier(n_estimators=200, random_state=0)
fr_clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=0)

In [18]:
clf_pred = fr_clf.predict(X_test)

In [19]:
fr_clf.score(X_test,y_test)

0.9171270718232044

In [20]:
print(classification_report(y_test,clf_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91       615
           1       0.92      0.92      0.92       652

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267



# 3. Decision Tree Classification

In [21]:
from sklearn.tree import DecisionTreeClassifier

In [22]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X_train,y_train)

DecisionTreeClassifier()

In [23]:
dt_pred = dt_clf.predict(X_test)

In [24]:
dt_clf.score(X_test,y_test)

0.8058405682715075

In [25]:
print(classification_report(y_test,dt_pred))

              precision    recall  f1-score   support

           0       0.82      0.78      0.79       615
           1       0.80      0.83      0.82       652

    accuracy                           0.81      1267
   macro avg       0.81      0.80      0.81      1267
weighted avg       0.81      0.81      0.81      1267



# 4. Support Vector machine

In [26]:
from sklearn.svm import SVC

In [27]:
svm_clf = SVC(random_state=0)
svm_clf.fit(X_train,y_train)

SVC(random_state=0)

In [28]:
svm_pred = svm_clf.predict(X_test)

In [29]:
svm_clf.score(X_test,y_test)

0.9226519337016574

In [30]:
print(classification_report(y_test,svm_pred))

              precision    recall  f1-score   support

           0       0.92      0.92      0.92       615
           1       0.93      0.92      0.92       652

    accuracy                           0.92      1267
   macro avg       0.92      0.92      0.92      1267
weighted avg       0.92      0.92      0.92      1267



# 5. K Nearest Neighbors

In [31]:
from sklearn.neighbors import KNeighborsClassifier

In [32]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_test,y_test)

KNeighborsClassifier()

In [33]:
knn_pred = knn_clf.predict(X_test)

In [34]:
knn_clf.score(X_test,y_test)

0.6393054459352802

In [35]:
print(classification_report(y_test,knn_pred))

              precision    recall  f1-score   support

           0       0.57      1.00      0.73       615
           1       0.99      0.30      0.46       652

    accuracy                           0.64      1267
   macro avg       0.78      0.65      0.60      1267
weighted avg       0.79      0.64      0.59      1267

